# Load package

In [1]:
%matplotlib inline

# General packages for system, time, etc
import os, time, csv, sys
import datetime
from datetime import date
import glob

# scitnific computing and plotting
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

# HDDM related packages
import pymc as pm
import hddm
import kabuki
import arviz as az
print("The current HDDM version is: ", hddm.__version__)
print("The current kabuki version is: ", kabuki.__version__)
print("The current PyMC version is: ", pm.__version__)
print("The current ArviZ version is: ", az.__version__)

# parallel processing related
from p_tqdm import p_map
from functools import partial

from sklearn.metrics import r2_score

The current HDDM version is:  0.9.8RC
The current kabuki version is:  0.6.5RC3
The current PyMC version is:  2.3.8
The current ArviZ version is:  0.14.0


# Load data

In [2]:
cpath=os.getcwd()
lname=cpath+'/ior_stroop_alldata.csv'
data=pd.read_csv(lname)
data['rt']=data['rt']/1000
data.rename(columns={"sub_idx": "subj_idx"}, inplace=True)

In [3]:
def m1_id(isub,csvname,dbname,mname,df=None, samples=10000, burn=2000, thin=2): 
    
    print('running sub: {:d}'.format(isub+1))
     
    m      = hddm.HDDM(df, depends_on = {'v':'expconds','a':'expconds','t':'expconds'},include={'v','a','t','sv','st'},
                     p_outlier = .05,is_group_model=False,bias=False)
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle')
    m.save(mname)
    res = pd.DataFrame([m.values])
    res.to_csv(csvname)
    return m

In [ ]:

for isub in range(len(set(data['subj_idx']))):
    
    thisname='ior_stroop_m1individual_w'+str(isub+1)
    csvname=thisname+'.csv'
    dbname=thisname+'.db'
    mname=thisname
    
    thisdata=data[data['subj_idx']==isub+1]
    
    m1_id(isub,csvname,dbname,mname,thisdata)